<a href="https://colab.research.google.com/github/pycriador/PyMalaDireta/blob/main/%5BGoogle_Drive%5D_PyMalaDireta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Projeto de Fevereiro


Um programa simples para criar seus arquivos de mala direta utilizando um template em XLSX (Excel) e um documento DOCX (Word).

Primeiro, vamos instalar as dependências do Python? Clique no Play e aguarde.

In [ ]:
!pip install docxtpl pandas numpy

Agora, vamos alocar o seu Google Drive para salvar os arquivos em DOCX e se você desejar, os arquivos PDF. Clique no Play, aceite o script conectar no seu Google Drive e aguarde a criação das pastas.

O script vai criar uma pasta chamada docx e pdf no seu Google Drive, você pode deletar essas pastas a qualquer momento.

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

#Criar diretório para exportação
!mkdir "/content/drive/MyDrive/docx"
!mkdir "/content/drive/MyDrive/pdf"

Agora vamos baixar o LibreOffice portátil. Ele será utilizado para converter os documentos DOCX em PDF. Se você não quiser converter os documentos gerados em PDF é só ignorar esse passo.

In [ ]:
import os.path

if os.path.isfile('/content/drive/MyDrive/LibreOffice-still.basic-x86_64.AppImage'):
  print('LibreOffice Portátil já está no seu Google Drive')
else:
  !wget https://libreoffice.soluzioniopen.com/stable/still/LibreOffice-still.basic-x86_64.AppImage -P /content/drive/MyDrive
  !chmod +x /content/drive/MyDrive/LibreOffice-still.basic-x86_64.AppImage

Esse passo é para importar uma classe que eu coloco nos meus projetos para localizar em arquivos em uma pasta. A implementação dessa classe nesse script é bem mais simples, mas ele aceita expressão regular e vários subdiretórios.

In [ ]:
import os, re

class appCORE():

    def files(self, path, extension):
        for file in os.listdir(path):
            if os.path.isfile(os.path.join(path, file)):
                if re.search(extension, file):
                    yield file

    def dir_list(self, path=None, extension=None):
        list = []

        for path_info in path:
            dirs = self.files(path_info, extension)
            for dir_info in dirs:
                list.append("%s/%s" % (path_info, dir_info))

        return list

Essa classe é a que vai fazer todo o trabalho, você poderá customizar alguns campos se desejar, por exemplo, a classe pega do arquivo XLSX uma coluna chamada "Nome" e "CPF", se você não tem esses campos na sua planilha, você poderá mudar o nome do arquivo final.

In [ ]:
from docxtpl import DocxTemplate
import pandas as pd
import numpy as np
import time
import subprocess

class appMalaDireta():

    def read_template(self, templates='templates', page=None, encoding='ISO-8859-1'):
        self.template = DocxTemplate(f"{templates}/{page}")

    def read_xlsx(self, file=None, sheet=None):
        xlsx = pd.ExcelFile(file)
        return pd.read_excel(xlsx)

    def create(self, page='', export_name='', templates_path='', encoding='', 
               data=pd.DataFrame(), export_docx=''):
        for index, row in data.iterrows():
            timestamp = int(time.time())
            #Ler HTML como template
            self.read_template(templates=templates_path, page=page, encoding=encoding)
            #### Você poderá mudar o nome do arquivo final aqui
            #### Coloque entre {row['COLUNA']} o nome da coluna do seu Excel
            #### Se você não sabe o que fazer nessa parte, e seu arquivo não tem
            #### uma coluna NOME e CPF, só #comente as duas próximas linhas
            CPF = ''.join(filter(str.isdigit, row["CPF"]))
            export_name = f'{row["NOME"].upper().strip()}-{CPF}'
            #print(f'Gerando o arquivo: {export_name}')
            #Converter Nan em nada
            dados = row.replace(np.nan, '', regex=True)
            #Converter dados em string
            dados = dados.apply(str)
            self.template.render(dados.str.upper())
            print(f'Gerando o DOCX: {export_docx}/{export_name}-{timestamp}.docx')
            self.template.save(f'{export_docx}/{export_name}-{timestamp}.docx')

Agora chegou a hora de escolher os arquivos para mala direta. Primeiro, você escolherá o arquivo com a base de nomes, no formato XLSX

In [ ]:
from google.colab import files
print('Escolha o arquivo com os dados em XLSX')
ap_up = files.upload()

for ap in ap_up.keys():
  app = appMalaDireta()
  base = app.read_xlsx(file=ap)

Agora, você escolherá o template em DOCX para a carta. Nessa parte, o próprio programa vai preencher o template com as informações da base em Excel e gerar um DOCX para cada registro.
Não se esqueça, nesse arquivo os campos que você quer substituir com os valores do arquivo em Excel devem estar assim {{NOME}}.

Ex. Na planilha está com a coluna NOME, no template onde deve aparecer o nome deve estar assim: {{NOME}}

In [ ]:
print('Escolha o arquivo com o template em DOCX')
td_up = files.upload()
for td_up in td_up.keys():
  app.create(export_name='', templates_path='.', page=td_up, encoding='ISO-8859-1', data=base, export_docx='/content/drive/MyDrive/docx')

Esse passo é opcional, caso você tenha baixado o LibreOffice aqui no Colab e queira salvar os documentos em PDF. Os arquivos gerados serão salvos no seu Google Drive na pasta **pdf**.

In [ ]:
import subprocess

app = appCORE()

dir_list = []

path = ['/content/drive/MyDrive/docx']
extension = '.docx$'

for path_info in path:
    dirs = app.files(path_info, extension)
    for dir_info in dirs:
        dir_list.append("%s/%s" % (path_info, dir_info))

if len(dir_list) > 0:
    for ffile in dir_list:
      try:
        subprocess.check_call([f'/content/drive/MyDrive/LibreOffice-still.basic-x86_64.AppImage', '--headless', '--norestore',
                            '--writer', '--convert-to', 'pdf', '--outdir', '/content/drive/MyDrive/pdf', f'{ffile}'])
      except Exception as e:
        print(e)